# **Sprint 9 Showz analysis**

## Paso 1. Acceda los datos y prepáralos para el análisis

Almacena los datos de visitas, pedidos y gastos en variables.  Optimiza los datos para el análisis. Asegúrate de que cada columna contenga el tipo de datos correcto. 




In [1]:
# cargar librerías
import pandas as pd 
import numpy as np
import re 


In [2]:
# importar datos
visits = pd.read_csv('https://code.s3.yandex.net/datasets/visits_log_us.csv')
print(visits.head())
print('-------')
orders = pd.read_csv('https://code.s3.yandex.net/datasets/orders_log_us.csv')
print(orders.head())
print('-------')
costs = pd.read_csv('https://code.s3.yandex.net/datasets/costs_us.csv')
print(costs.head())
print('-------')

    Device               End Ts  Source Id             Start Ts  \
0    touch  2017-12-20 17:38:00          4  2017-12-20 17:20:00   
1  desktop  2018-02-19 17:21:00          2  2018-02-19 16:53:00   
2    touch  2017-07-01 01:54:00          5  2017-07-01 01:54:00   
3  desktop  2018-05-20 11:23:00          9  2018-05-20 10:59:00   
4  desktop  2017-12-27 14:06:00          3  2017-12-27 14:06:00   

                    Uid  
0  16879256277535980062  
1    104060357244891740  
2   7459035603376831527  
3  16174680259334210214  
4   9969694820036681168  
-------
                Buy Ts  Revenue                   Uid
0  2017-06-01 00:10:00    17.00  10329302124590727494
1  2017-06-01 00:25:00     0.55  11627257723692907447
2  2017-06-01 00:27:00     0.37  17903680561304213844
3  2017-06-01 00:29:00     0.55  16109239769442553005
4  2017-06-01 07:58:00     0.37  14200605875248379450
-------
   source_id          dt  costs
0          1  2017-06-01  75.20
1          1  2017-06-02  62.25
2    

### Examinar visits

In [3]:
# Examine visits
visits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 359400 entries, 0 to 359399
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   Device     359400 non-null  object
 1   End Ts     359400 non-null  object
 2   Source Id  359400 non-null  int64 
 3   Start Ts   359400 non-null  object
 4   Uid        359400 non-null  uint64
dtypes: int64(1), object(3), uint64(1)
memory usage: 13.7+ MB


In [4]:
visits['Device'].value_counts()

Device
desktop    262567
touch       96833
Name: count, dtype: int64

In [5]:
visits['Source Id'].value_counts()

Source Id
4     101794
3      85610
5      66905
2      47626
1      34121
9      13277
10     10025
7         36
6          6
Name: count, dtype: int64

### Limpiar visits
1. Columnas a minúsculas
2. End Ts y Start Ts a formato fecha

In [6]:
# Función para limpiar columnas
def clean_col_names(df):
    df = df.copy()
    df.columns = [
        re.sub(r'\W+', '_', col).strip('_').lower()
        for col in df.columns
    ]
    return df


In [7]:
# Limpiar visits.columns
visits = clean_col_names(visits)
visits

,device,end_ts,source_id,start_ts,uid
0,touch,2017-12-20 17:38:00,4,2017-12-20 17:20:00,16879256277535980062
1,desktop,2018-02-19 17:21:00,2,2018-02-19 16:53:00,104060357244891740
2,touch,2017-07-01 01:54:00,5,2017-07-01 01:54:00,7459035603376831527
3,desktop,2018-05-20 11:23:00,9,2018-05-20 10:59:00,16174680259334210214
4,desktop,2017-12-27 14:06:00,3,2017-12-27 14:06:00,9969694820036681168
...,...,...,...,...,...
359395,desktop,2017-07-29 19:07:19,2,2017-07-29 19:07:00,18363291481961487539
359396,touch,2018-01-25 17:38:19,1,2018-01-25 17:38:00,18370831553019119586
359397,desktop,2018-03-03 10:12:19,4,2018-03-03 10:12:00,18387297585500748294
359398,desktop,2017-11-02 10:12:19,5,2017-11-02 10:12:00,18388616944624776485


In [8]:
# Cambiar end_ts y start_ts a formato fecha
visits.end_ts = pd.to_datetime(visits['end_ts'])
visits.start_ts = pd.to_datetime(visits['start_ts'])
print(visits.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 359400 entries, 0 to 359399
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   device     359400 non-null  object        
 1   end_ts     359400 non-null  datetime64[ns]
 2   source_id  359400 non-null  int64         
 3   start_ts   359400 non-null  datetime64[ns]
 4   uid        359400 non-null  uint64        
dtypes: datetime64[ns](2), int64(1), object(1), uint64(1)
memory usage: 13.7+ MB
None


In [9]:
visits.sample(10)

,device,end_ts,source_id,start_ts,uid
257494,touch,2017-10-12 23:54:00,5,2017-10-12 23:50:00,3788838973332277912
259346,desktop,2018-02-28 23:02:00,4,2018-02-28 22:44:00,14888237163979894094
80560,desktop,2017-12-08 00:19:00,2,2017-12-08 00:12:00,1829736435757276287
233649,desktop,2018-03-25 15:00:00,9,2018-03-25 14:56:00,17786796508613183016
130578,desktop,2017-12-11 16:07:00,1,2017-12-11 15:53:00,15087669361906245158
149422,touch,2018-04-30 17:20:00,1,2018-04-30 16:55:00,337126187358259420
330527,desktop,2018-05-05 15:12:00,3,2018-05-05 14:57:00,6166747268563050393
359255,desktop,2017-08-05 01:03:19,5,2017-08-05 01:03:00,15250499413999327662
202010,desktop,2018-05-28 13:39:00,3,2018-05-28 13:29:00,13442759219179233185
22503,desktop,2018-01-22 14:12:00,2,2018-01-22 13:22:00,9737640335185488211


### Examinar orders

In [10]:
# Examine orders
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50415 entries, 0 to 50414
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Buy Ts   50415 non-null  object 
 1   Revenue  50415 non-null  float64
 2   Uid      50415 non-null  uint64 
dtypes: float64(1), object(1), uint64(1)
memory usage: 1.2+ MB


In [11]:
orders['Revenue'].describe()

count    50415.000000
mean         4.999647
std         21.818359
min          0.000000
25%          1.220000
50%          2.500000
75%          4.890000
max       2633.280000
Name: Revenue, dtype: float64

In [12]:
orders[orders.Revenue > 1000]

,Buy Ts,Revenue,Uid
23165,2017-12-10 13:04:00,1195.64,5539673724080479777
23244,2017-12-10 20:17:00,2633.28,5539673724080479777
24607,2017-12-17 18:06:00,1109.10,11149926373378902217
36522,2018-02-23 08:54:00,1236.28,11149926373378902217
36682,2018-02-24 09:25:00,1221.37,11149926373378902217
40386,2018-03-18 09:13:00,1073.11,11149926373378902217
47507,2018-05-16 19:53:00,1080.44,11149926373378902217


### Limpiar orders

1. limpiar nombres de columnas
2. 'Buy Ts' a formato fecha

In [13]:
orders = clean_col_names(orders)
orders.buy_ts = pd.to_datetime(orders.buy_ts)
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50415 entries, 0 to 50414
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   buy_ts   50415 non-null  datetime64[ns]
 1   revenue  50415 non-null  float64       
 2   uid      50415 non-null  uint64        
dtypes: datetime64[ns](1), float64(1), uint64(1)
memory usage: 1.2 MB


### Examinar costs

In [14]:
# Examine costs
costs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2542 entries, 0 to 2541
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   source_id  2542 non-null   int64  
 1   dt         2542 non-null   object 
 2   costs      2542 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 59.7+ KB


In [15]:
costs.sample(5)

,source_id,dt,costs
1363,4,2018-03-02,40.49
1544,5,2017-09-01,119.64
1599,5,2017-10-26,158.50
860,3,2017-10-13,703.26
2297,10,2017-09-27,17.71


In [16]:
costs.source_id.value_counts()

source_id
5     364
1     363
2     363
3     363
4     363
9     363
10    363
Name: count, dtype: int64

### Limpiar costs

1. 'dt' a formato fecha

In [17]:
costs.dt = pd.to_datetime(costs.dt)
costs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2542 entries, 0 to 2541
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   source_id  2542 non-null   int64         
 1   dt         2542 non-null   datetime64[ns]
 2   costs      2542 non-null   float64       
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 59.7 KB


## Paso 2. Haz informes y calcula métricas 

### Visitas:
- ¿Cuántas personas lo usan cada día, semana y mes?
- ¿Cuántas sesiones hay por día? (Un usuario puede tener más de una sesión).
- ¿Cuál es la duración de cada sesión?
- ¿Con qué frecuencia los usuarios regresan?






In [18]:
# Primero sacaremos de las fechas, el día, la semana, mes y año de la visita
visits['day'] = visits['start_ts'].dt.weekday
visits['week'] = visits['start_ts'].dt.isocalendar().week
visits['month'] = visits['start_ts'].dt.month
visits['year'] = visits['start_ts'].dt.isocalendar().year
visits.sample(10)

,device,end_ts,source_id,start_ts,uid,day,week,month,year
264919,desktop,2017-12-15 15:31:00,4,2017-12-15 15:19:00,15320517278030087556,4,50,12,2017
182342,desktop,2018-03-15 18:47:00,5,2018-03-15 18:19:00,15933795970653493810,3,11,3,2018
269497,desktop,2018-01-21 13:05:00,5,2018-01-21 13:03:00,6627893064446800297,6,3,1,2018
92505,desktop,2018-03-09 21:00:00,4,2018-03-09 20:47:00,11011220864619881430,4,10,3,2018
275259,desktop,2018-03-26 10:30:00,1,2018-03-26 10:22:00,774976817171856731,0,13,3,2018
329661,desktop,2018-03-28 06:58:00,3,2018-03-28 06:49:00,4631934393540629724,2,13,3,2018
277438,touch,2017-08-21 12:09:00,5,2017-08-21 12:03:00,8693221690747963305,0,34,8,2017
162495,desktop,2017-08-17 12:26:00,9,2017-08-17 12:26:00,3333945287425016364,3,33,8,2017
357081,desktop,2017-11-05 14:35:00,5,2017-11-05 14:21:00,5326980333433499848,6,44,11,2017
356041,desktop,2018-03-07 12:32:00,3,2018-03-07 11:54:00,3961221463537920802,2,10,3,2018


In [19]:
#Promedio de visitas por día

by_days = visits.groupby(['week', 'day'])['uid'].nunique().reset_index()
print(f'Al día, en promedio, se tienen {by_days.uid.mean():,.2f} visitas.')


Al día, en promedio, se tienen 910.46 visitas.


In [20]:
#Promedio de visitas por semana

by_week = visits.groupby(['year', 'week'])['uid'].nunique().reset_index()
print(f'Por semana, en promedio, tenemos {by_week.uid.mean():,.2f} visitas')

Por semana, en promedio, tenemos 5,716.25 visitas


In [21]:
#Promedio de visitas por semana

by_month = visits.groupby(['year', 'month'])['uid'].nunique().reset_index()
print(f'Por mes, en promedio, tenemos {by_month.uid.mean():,.2f} visitas')

Por mes, en promedio, tenemos 23,228.42 visitas


In [22]:
#Promedio de sesiones por día

sessions_per_day = visits.groupby(['week', 'day'])['uid'].count().reset_index()
print(f'Al día, en promedio, se tienen {sessions_per_day.uid.mean():,.2f} sesiones.')

Al día, en promedio, se tienen 990.08 sesiones.


In [23]:
# Duración de sesiones

visits['session_duration'] = visits.end_ts - visits.start_ts
visits['session_duration_secs'] = visits.session_duration.dt.total_seconds()
avg_session_duration = visits.session_duration_secs.median()
print(f'La media de la duración de la sesión es de {avg_session_duration/60:,.2f} minutos.')

La media de la duración de la sesión es de 5.00 minutos.


In [24]:
# Con qué frecuencia los usuarios regresan? 
comeback_frec = visits.sort_values(by=['uid', 'start_ts'])
visits['time_since_last_visit'] = comeback_frec.groupby(['uid'])['start_ts'].diff()
visits['days_between_visits'] = visits.time_since_last_visit.dt.days
ave_return_time = visits.days_between_visits.median()
print(f'La media del tiempo en que suelen regresar los usuarios es de {ave_return_time:.2f} días.')


La media del tiempo en que suelen regresar los usuarios es de 4.00 días.


En la duración de la sesión y en el tiempo en que suelen regresar los usuarios usamos **la media**, ya que en el promedio los valores atípicos elevaban las cantidades.

### Ventas:
- ¿Cuándo empieza la gente a comprar? (En el análisis de KPI, generalmente nos interesa saber el tiempo que transcurre entre el registro y la conversión, es decir, cuando el usuario se convierte en cliente. Por ejemplo, si el registro y la primera compra ocurren el mismo día, el usuario podría caer en la categoría Conversion 0d. Si la primera compra ocurre al día siguiente, será Conversion 1d. Puedes usar cualquier enfoque que te permita comparar las conversiones de diferentes cohortes para que puedas determinar qué cohorte o canal de marketing es más efectivo.)
- ¿Cuántos pedidos hacen durante un período de tiempo dado?
- ¿Cuál es el tamaño promedio de compra?
- ¿Cuánto dinero traen? (LTV)

In [25]:
# Fechas de primeras compras
purchases = orders[orders.revenue > 0]
first_purchase = purchases.groupby('uid')['buy_ts'].min().reset_index()
first_purchase.columns = ['uid', 'first_buy_ts']
orders = pd.merge(orders, first_purchase, on='uid')
orders.sample(5)



,buy_ts,revenue,uid,first_buy_ts
10672,2017-10-04 21:43:00,1.34,8483834426764801902,2017-10-04 21:43:00
991,2017-06-08 14:35:00,1.47,15724911357977965472,2017-06-08 12:36:00
17787,2017-11-16 11:04:00,11.61,15353005119064919638,2017-11-16 11:04:00
7087,2017-09-07 13:02:00,2.44,3022298465865324289,2017-09-07 13:02:00
3655,2017-07-17 13:25:00,1.83,14176717271193604114,2017-06-22 16:51:00


In [26]:
# Fechas de registros
registration = orders.groupby('uid')['buy_ts'].min().reset_index()
registration.columns = ['uid', 'registration_ts']
orders = pd.merge(orders, registration, on='uid')
orders.sample(5)

,buy_ts,revenue,uid,first_buy_ts,registration_ts
30165,2018-01-20 09:54:00,9.17,3450911796645934534,2018-01-20 09:54:00,2018-01-20 09:54:00
27355,2017-12-31 14:34:00,1.34,2228294324265431661,2017-12-31 14:34:00,2017-12-31 14:34:00
4880,2017-08-04 17:08:00,1.22,13888745432979765063,2017-06-05 11:39:00,2017-06-05 11:39:00
16311,2017-11-06 08:18:00,4.28,11751156543780332305,2017-11-06 08:18:00,2017-11-06 08:18:00
10241,2017-10-03 10:30:00,0.73,15313242664806867428,2017-10-03 10:30:00,2017-10-03 10:30:00


In [27]:
# Crear cohortes por días transcurridos para comprar despúes del registro
orders['days_to_purchase'] = (orders['first_buy_ts'] - orders['registration_ts']).dt.days
orders.sample(5)

,buy_ts,revenue,uid,first_buy_ts,registration_ts,days_to_purchase
47464,2018-05-16 18:32:00,0.24,13985976893008941167,2017-09-17 21:03:00,2017-09-17 21:03:00,0
36156,2018-02-21 14:50:00,8.80,10343016064897450067,2017-12-07 13:58:00,2017-12-07 13:58:00,0
46847,2018-05-10 15:32:00,1.40,17587536129601095913,2018-05-10 12:36:00,2018-05-10 12:36:00,0
16785,2017-11-09 13:31:00,2.44,14525704262455462265,2017-07-31 16:42:00,2017-07-31 16:42:00,0
20513,2017-11-27 18:24:00,3.05,7973565592939705006,2017-11-27 18:24:00,2017-11-27 18:24:00,0


In [28]:
orders['cohort_label'] = orders['days_to_purchase'].apply(lambda x: f'{x}d')
orders.sample(5)

,buy_ts,revenue,uid,first_buy_ts,registration_ts,days_to_purchase,cohort_label
12637,2017-10-13 22:18:00,9.17,304280166509818216,2017-10-13 22:18:00,2017-10-13 22:18:00,0,0d
13690,2017-10-20 11:13:00,3.67,6122245434183457733,2017-10-20 11:13:00,2017-10-20 11:13:00,0,0d
37941,2018-03-03 13:00:00,7.00,9603173203015373254,2018-03-03 13:00:00,2018-03-03 13:00:00,0,0d
41402,2018-03-23 18:58:00,0.49,5475544912668909685,2018-03-23 00:04:00,2018-03-23 00:04:00,0,0d
7942,2017-09-16 08:07:00,9.17,2153471313064870024,2017-09-16 08:07:00,2017-09-16 08:07:00,0,0d


In [29]:
by_cohorts = orders.groupby('cohort_label')['uid'].nunique()
by_cohorts

cohort_label
0d      36504
117d        1
12d         1
167d        1
3d          1
65d         1
Name: uid, dtype: int64

Practicamente el 99% de los usuarios compran el mismo día que se registraron. 

Probablemente no sabían de la aplicación, ni les interesaba registrarse hasta que quisieron comprar boletos para un evento al que sí querían ir.

## Marketing:
 1. ¿Cuánto dinero se gastó?  (Total/por fuente de adquisición/a lo largo del tiempo) 
 2. ¿Cuál fue el costo de adquisición de clientes de cada una de las fuentes?
 3. ¿Cuán rentables eran las inversiones? (ROMI)
Traza gráficos para mostrar cómo difieren estas métricas para varios dispositivos y fuentes de anuncios y cómo cambian con el tiempo. 